In [470]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing

%matplotlib inline

In [471]:
def submit(preds, test_feature, label_map):
    submission = test_feature[['id']]
    submission['pred'] = [label_map[l] for l in np.nditer(preds)]
    submission.to_csv('./predicts/0216_1_partition.csv', header=False, index=False)
    return submission
    

In [472]:
test_feature = pd.read_csv('./test_feature_line_index_0217.csv',header=None)
test_feature.head()
train_feature = pd.read_csv('./train_feature_line_index_0217.csv',header=None)
test_feature.rename(columns={0:'id'}, inplace=True)
train_feature.head()


,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,696220,2807192.6,1206.6,4833.9,-362.9,3137136.9,4.0,2326.5,-0.3,380.9,...,1.5,419.6,3142.2,3516.1,2697.3,109977.9,1.1,0.1,0.9,star
1,911466,899.3,112.2,166.7,-2.9,291801.1,1.5,8.0,-0.0,1.0,...,0.8,1.1,106.6,125.3,82.4,3730.8,1.2,0.0,0.8,star
2,915856,434.2,61.4,126.5,24.1,159725.3,2.1,7.1,0.4,0.2,...,0.8,0.1,81.3,88.6,76.2,2845.1,1.1,0.0,0.9,star
3,720026,17101718.7,12073.5,20181.5,3666.1,31391013.8,1.7,1416.5,0.3,4507.3,...,0.9,4272.4,16433.0,17430.5,14962.0,575156.0,1.1,0.3,0.9,star
4,1198160,11729.1,379.8,491.7,-225.7,987376.9,1.3,30.9,-0.6,10.4,...,0.9,10.6,359.2,410.2,273.5,12572.3,1.1,0.0,0.8,star


In [473]:
train_feature.shape[1]

122

In [480]:
columns = {0:'id'}
columns[train_feature.shape[1]-1] = 'label'
train_feature.rename(columns=columns, inplace=True)
train_feature.head()
train_feature = train_feature.fillna(0)

In [481]:
def getTrain(train_feature):
    train_feature = train_feature.sample(train_feature.shape[0],replace=False)
    testEnd = int(train_feature.shape[0]/3)
    train_test = train_feature.loc[:testEnd,:]
    train_feature = train_feature.loc[testEnd:,:]
    return (train_test, train_feature)

In [482]:
def balanceData(train_feature, realBalance=False):
    if realBalance:
        train_star = train_feature[train_feature.label=='star'].sample(30000, replace=True)
        unknow_star = train_feature[train_feature.label=='unknown'].sample(30000, replace=True)
        galaxy_star = train_feature[train_feature.label=='galaxy'].sample(30000, replace=True)
        qso_star = train_feature[train_feature.label=='qso'].sample(30000, replace=True)
        sampled_train = pd.concat([train_star,unknow_star,galaxy_star, qso_star])
    else :
        sampled_train = train_feature
    features = train_feature.columns[1:-1]
    x= sampled_train[features]
    y, label_map = pd.factorize(sampled_train['label'])
    return x, y, label_map

In [483]:
train_t, train_f =  getTrain(train_feature)
x, y, label_map = balanceData(train_f)
t_x, t_y, _ = balanceData(train_t, realBalance=True)

In [484]:
clf = RandomForestClassifier(n_jobs=4, max_depth=30, n_estimators=200)
# clf = svm.SVC(kernel='rbf')

In [485]:
rs = clf.fit(x, y)
# preds_test = clf.predict(t_x)
clf.score(t_x, t_y, )

0.28230833333333333

In [455]:
scaler = preprocessing.Normalizer().fit(x)
p_x = scaler.transform(x)  
p_t_x = scaler.transform(t_x)  
rs = clf.fit(p_x, y)
preds_test = clf.predict(p_t_x)
clf.score(p_t_x, t_y)

0.47013333333333335

In [315]:
preds = clf.predict(test_feature.iloc[:,1:])


In [140]:
sb = submit(preds, test_feature, label_map)
sb.pred.value_counts()

/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


star       84091
unknown    12504
galaxy      2454
qso          951
Name: pred, dtype: int64

In [108]:
sb.pred.value_counts()

star       84431
unknown    12243
galaxy      2283
qso         1043
Name: pred, dtype: int64